### try to cluster lng/lat using `sklearn.cluster.KMeans`

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [6]:
# 경로 변수 (고정)

dataLoc = '../_data/'

trainfile = 'train_03_997.csv'
old_trainfile = 'train.csv'
kaggle_trainfile = 'train_k.csv'

testfile = 'test_clean.csv'
old_testfile = 'test.csv'

train_171016 = '171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'
train_dropped_nan_fixed = '../_data/train__dropped__no_disaster_nan_fixed.csv'
train_added_airps = '../_data/train__dropped__lnglat_saved__airports__no_disaster.csv'
train_added_airps_cleaned = '../_data/train__dropped__cleaned__airports__no_disaster.csv'
train_added_airps_disas_cleaned = '../_data/train__dropped__cleaned__eval__airports__disaster.csv'
train_added_weekhr = 'train__dropped__cleaned__eval__a_d_weekhour.csv'
train_added_manh = '../_data/train__dropped__air_bliz_weekhr_manh.csv'
train_all_vars_rough = '../_data/train__all_vars__rough.csv'
train_dropped_cooks = '../_data/train__dropped_cooks__no_airp__no_blizz__no_weekhour__.csv'
train_latest = train_dropped_cooks

In [3]:
pd.options.display.max_columns = 40

In [7]:
# train_with_osrm_and_weather.csv
df_train = pd.read_csv(dataLoc + train_latest)
df_train.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [5]:
df_train.shape

(683021, 27)

In [3]:
df_copy = df_train.copy()

### 10/19

## cross validation

In [111]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split


cv = KFold(10)

scores = np.zeros(10)
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(main_df.iloc[:, :-1], main_df.iloc[:, -1], test_size=0.33)
    df_train = pd.concat([X_train, y_train], axis=1)
    print("trial {}".format(i+1))
    model_train = sm.OLS.from_formula("dur ~ C(dow) + C(hr) + L2 - 1", data=df_train)
    res_train = model_train.fit()
    y_pred = res_train.predict(X_test)
    y_pred[y_pred < 0] = 0
    try:
        scores[i] = mean_squared_log_error(y_test, y_pred)
        print(pd.concat([y_test, y_pred], axis=1).head(3))
    except:
        print("except")
        print(y_test[y_test<=0].shape)
        print(y_pred[y_pred<=0].shape)
        break

np.sqrt(scores)

trial 1
        dur           0
32056   722  655.530932
246647  228  497.563814
580570  327  438.177414
trial 2
         dur            0
402392   243   523.023825
476450   631   802.832409
641728  1254  1158.212300
trial 3
         dur            0
567080   936   740.242101
477970   213   496.391931
450949  2026  3175.998947
trial 4
         dur            0
141933   290   613.255237
62897   2147  1266.403282
616943   338   572.932548
trial 5
        dur           0
365482  288  526.312419
243243  603  459.834124
558105  321  515.614850
trial 6
        dur           0
290990  351  562.388685
346842  485  563.731603
546194  473  658.687985
trial 7
         dur            0
651666   475   788.173120
465130   380   579.273924
109428  1348  1588.213578
trial 8
        dur           0
588593  555  565.236320
627001  311  384.453169
242250  592  477.654228
trial 9
        dur           0
461343  852  827.210865
573278  702  645.555302
497704  569  778.201680
trial 10
        dur           0

array([ 0.5028246 ,  0.50288057,  0.50263951,  0.50303881,  0.50258708,
        0.50204513,  0.50485645,  0.50465234,  0.50294299,  0.5034325 ])

### 10/20

In [23]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

fmla = 'dur1 ~ C(is_hol) + C(dow):C(hr) + scale(total_dist)\
             + scale(day_max_temp) + scale(day_rainfall)\
             + scale(day_snowfall) + scale(day_snowdepth)\
             + scale(hr_temp) + scale(hr_humid) + scale(hr_windspeed) - 1'

cv = KFold(10)

scores = np.zeros(10)
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(df_copy.iloc[:, :-1], df_copy.iloc[:, -1], test_size=0.33)
    df_train = pd.concat([X_train, y_train], axis=1)
    print("trial {}".format(i+1))
    model_train = sm.OLS.from_formula(fmla, data=df_train)
    res_train = model_train.fit()
    y_pred = res_train.predict(X_test)
    y_pred[y_pred < 0] = 0
    try:
        scores[i] = mean_squared_log_error(y_test, y_pred)
        print(pd.concat([y_test, y_pred], axis=1).head(3))
    except:
        print("except")
        print(y_test[y_test<=0].shape)
        print(y_pred[y_pred<=0].shape)
        break

np.sqrt(scores)

trial 1
        dur1            0
251643  3161  3114.350900
353661  1016   960.782166
74115    473   535.849719
trial 2
        dur1           0
350285  1084  799.577067
135916   163  539.587685
405686   360  429.496466
trial 3
        dur1           0
584634   177  575.211074
237392   408  617.629919
606171   738  855.881578
trial 4


KeyboardInterrupt: 

In [24]:
np.sqrt(scores)

array([ 0.47922921,  0.47979153,  0.48086002,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])